In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tqdm import tqdm

In [2]:
# load dataset
mnist = input_data.read_data_sets("MNIST", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size


# we need name space to add summary
with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 784])
    y = tf.placeholder(tf.float32, [None, 10])
    keep_prob = tf.placeholder(tf.float32)

with tf.variable_scope("layer1"):
    W_L1 = tf.Variable(tf.truncated_normal([784,200],stddev=0.1)) # initialization is important here
    b_L1 = tf.Variable(tf.truncated_normal([200],stddev=0.1))
    o_L1 = tf.nn.relu(tf.matmul(x,W_L1)+b_L1)
    d_L1 = tf.nn.dropout(o_L1, keep_prob)
with tf.variable_scope("layer2"):
    W_L2 = tf.Variable(tf.truncated_normal([200,50],stddev=0.1))
    b_L2 = tf.Variable(tf.truncated_normal([50],stddev=0.1))
    o_L2 = tf.nn.relu(tf.matmul(d_L1,W_L2)+b_L2)
    d_L2 = tf.nn.dropout(o_L2, keep_prob)
with tf.variable_scope("layer3"):
    W_L3 = tf.Variable(tf.truncated_normal([50,10],stddev=0.1))
    b_L3 = tf.Variable(tf.truncated_normal([10],stddev=0.1))
    o_L3 = tf.nn.relu(tf.matmul(d_L2,W_L3)+b_L3)

# d_L3 = tf.nn.dropout(o_L3, keep_prob)
prediction = tf.nn.softmax(o_L3)

loss = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction)
update = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

for var in tf.trainable_variables():
    tf.summary.histogram("parameters/"+ var.op.name, var)

merged = tf.summary.merge_all()

init = tf.global_variables_initializer()

correct_prediction = tf.equal(tf.arg_max(y,1), tf.arg_max(prediction,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# softmax and sorftmax_cross_entropy_with_logits is a good pair
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter("logs", sess.graph) # write into it
    for epoch in range(1):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _merged, _ = sess.run([merged, update], feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.3})
#             print(sess.run(loss, feed_dict={x:batch_xs, y:batch_ys}))
        writer.add_summary(_merged, epoch)
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images,y:mnist.test.labels, keep_prob:0.3})
        print("Iter"+ str(epoch)+" Testing Accuracy "+ str(acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
Use `argmax` instead
Iter0 Testing Accuracy 0.6841


if the there's a lot of repeated graph, it's because of cache in Jupyter, so restart and clear all output